# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

\**ANSWERS FOR PART 1 IN THIS CELL*\*



## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

\**ANSWERS FOR PART 2 IN THIS CELL*\*

## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [1]:
from utils import *

In [45]:
# Utility functions

def get_layer_params(in_features, out_features, weight_init_sd=1.0):
    nd = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([weight_init_sd]))
    params = (nd.sample(sample_shape=torch.Size([in_features, out_features]))).squeeze()
    params.requires_grad_()
    return params

def add_bias(inputs):
    return torch.cat([inputs, torch.ones([inputs.shape[0], 1])], dim=1)

In [46]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier():
    def __init__(self, weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        self.params = get_layer_params(785, 10, weight_init_sd=weight_init_sd)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
                
    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **        
        #######################################################################    
        h = inputs.view(-1, 784)
        h = add_bias(h)
        return F.log_softmax(h @ self.params, dim=1)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.params]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return torch.sum(torch.sum(torch.abs(self.params), dim=1), dim=0)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return torch.sum(torch.sum(self.params ** 2, dim=1), dim=0)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [47]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
def run_with_model(lr=0.003, l1=0., l2=0.001, suppress_output=False, epochs=10, weight_init_sd=1.0):
    print("learning rate: ", lr, ", l2 coefficient: ", l2, " l1 coefficient: ", l1)
    model = MultinomialLogisticRegressionClassifier(weight_init_sd=weight_init_sd)
    res = run_experiment(
        model,
        optimizer=optim.Adam(model.parameters(), lr=lr),
        train_loader=train_loader_0,
        val_loader=val_loader_0,
        test_loader=test_loader_0,
        n_epochs=epochs,
        l1_penalty_coef=l1,
        l2_penalty_coef=l2,
        suppress_output=suppress_output
    )
    return res

# lr: 0.003, l2: 0.001, weight: 1.0, acc: 0.915
# Test set:	Average loss: 0.3061, Accuracy: 0.9160

lr = 0.003
l1 = 0.0
l2 = 0.001
weight = 1.0
epochs = 10

res = run_with_model(lr=lr, l2=l2, l1=l1, epochs=epochs, weight_init_sd=weight)

Learning rate  0.003  l2 coefficient  0.001
Epoch 0: training...
Train set:	Average loss: 0.4312, Accuracy: 0.8831
Validation set:	Average loss: 0.3328, Accuracy: 0.9090

Epoch 1: training...
Train set:	Average loss: 0.3331, Accuracy: 0.9099
Validation set:	Average loss: 0.3302, Accuracy: 0.9123

Epoch 2: training...
Train set:	Average loss: 0.3281, Accuracy: 0.9106
Validation set:	Average loss: 0.3242, Accuracy: 0.9100

Epoch 3: training...
Train set:	Average loss: 0.3268, Accuracy: 0.9110
Validation set:	Average loss: 0.3289, Accuracy: 0.9078

Epoch 4: training...
Train set:	Average loss: 0.3257, Accuracy: 0.9106
Validation set:	Average loss: 0.3150, Accuracy: 0.9108

Epoch 5: training...
Train set:	Average loss: 0.3264, Accuracy: 0.9101
Validation set:	Average loss: 0.3242, Accuracy: 0.9105

Epoch 6: training...
Train set:	Average loss: 0.3253, Accuracy: 0.9109
Validation set:	Average loss: 0.3245, Accuracy: 0.9113

Epoch 7: training...
Train set:	Average loss: 0.3254, Accuracy: 0.9

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [48]:
# *CODE FOR PART 4.1 IN THIS CELL*
class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        params_1 = get_layer_params(785, 128, weight_init_sd=weight_init_sd)
        params_2 = get_layer_params(129, 64, weight_init_sd=weight_init_sd)
        params_3 = get_layer_params(65, 32, weight_init_sd=weight_init_sd)
        params_4 = get_layer_params(33, 10, weight_init_sd=weight_init_sd)        
                
        self.params = [params_1, params_2, params_3, params_4]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        h = inputs.view(-1, 784)
                
        for indx, param in enumerate(self.parameters()):
            if indx == len(self.parameters()) - 1:
                break
            h = add_bias(h)
            h = self.activation(h @ param)
        
        h = add_bias(h)
        return F.log_softmax(h @ self.parameters()[3], dim=1)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return self.params
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        S = 0.
        
        for param in self.params:
            S += torch.sum(torch.sum(torch.abs(param), dim=1), dim=0)
        return S
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        S = 0.
        for param in self.params:
            S += torch.sum(torch.sum(param ** 2, dim=1), dim=0)
        return S    
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

In [53]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

# lr = 0.0043
# l1 = 0.0
# l2 = 0.001
# weight = 1.0
# func = 'relu'
# Test set:	Average loss: 0.1473, Accuracy: 0.9572

lr = 0.0043
l1 = 0.0
l2 = 0.001
weight = 1.0
func = 'relu'
# Test set:	Average loss: 0.1191, Accuracy: 0.9637

model = MultilayerClassifier(activation_fun=func, weight_init_sd=weight)
res = run_experiment(
    model,
    optimizer=optim.Adam(model.parameters(), lr),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=l1,
    l2_penalty_coef=l2,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.2800, Accuracy: 0.9144
Validation set:	Average loss: 0.2051, Accuracy: 0.9382

Epoch 1: training...
Train set:	Average loss: 0.1815, Accuracy: 0.9457
Validation set:	Average loss: 0.1681, Accuracy: 0.9505

Epoch 2: training...
Train set:	Average loss: 0.1669, Accuracy: 0.9493
Validation set:	Average loss: 0.1588, Accuracy: 0.9498

Epoch 3: training...
Train set:	Average loss: 0.1587, Accuracy: 0.9520
Validation set:	Average loss: 0.1591, Accuracy: 0.9530

Epoch 4: training...
Train set:	Average loss: 0.1527, Accuracy: 0.9529
Validation set:	Average loss: 0.1584, Accuracy: 0.9538

Epoch 5: training...
Train set:	Average loss: 0.1486, Accuracy: 0.9549
Validation set:	Average loss: 0.1590, Accuracy: 0.9525

Epoch 6: training...
Train set:	Average loss: 0.1471, Accuracy: 0.9553
Validation set:	Average loss: 0.1595, Accuracy: 0.9535

Epoch 7: training...
Train set:	Average loss: 0.1480, Accuracy: 0.9557
Validation set:	Average loss: 0.1880, Ac

\**ANSWERS FOR PART 4.3 IN THIS CELL*\*